# Step 1: Stimulate Rankings of Relevance for E and P (5 points)

In [45]:
import itertools as iter
import numpy as np
import random
import math
import pandas as pd

#The gradings
relevance_gradings = ['N', 'R', 'HR']

# Create all possible versions of length five rankings
list_p = [p for p in iter.product(relevance_gradings, repeat=5)]
list_e = [p for p in iter.product(relevance_gradings, repeat=5)]

print(len(list_e))

# Create all possible pairs of the two lists - if computing power permits
#list_pairs = [list(zip(list_p, p)) for p in itertools.permutations(list_e)]

# Sampling - take one at random from list 1 and one 1 from list 2 until 1000.
sample_size = 1000
sample_list_p = [random.choice(list_p) for _ in range(sample_size)]
sample_list_e = [random.choice(list_e) for _ in range(sample_size)]

sample_pairs = list(zip(sample_list_p, sample_list_e))

for i in range(0,5):
    print(sample_pairs[i])

243
(('N', 'R', 'HR', 'R', 'HR'), ('R', 'R', 'N', 'HR', 'R'))
(('R', 'N', 'R', 'R', 'R'), ('N', 'HR', 'N', 'HR', 'R'))
(('N', 'N', 'R', 'N', 'R'), ('R', 'R', 'R', 'HR', 'N'))
(('N', 'HR', 'R', 'HR', 'HR'), ('N', 'HR', 'N', 'N', 'N'))
(('N', 'R', 'R', 'HR', 'R'), ('HR', 'R', 'N', 'R', 'HR'))


# Step 2: Implement Evaluation Measures (15 points)

### Binary Measure - Average Precision

In [46]:
def calc_ave_precision(list_of_relevance):
    rank_length = len(list_of_relevance)
    running_score = 0
    relevant_count = 0
    for i in range(0, rank_length):
        if(list_of_relevance[i] == "R" or list_of_relevance[i] == "HR"):
            relevant_count += 1
            running_score += relevant_count / (i+1)
    ave_precision = running_score / rank_length
    return ave_precision

p_AP_scores = []
e_AP_scores = []

# Calculate average precision for all queries
for pairs in sample_pairs:
    p_AP_scores.append(calc_ave_precision(pairs[0]))
    e_AP_scores.append(calc_ave_precision(pairs[1]))

# Calculate the average of the average precisions across queries
p_ave_precision_over_queries = sum(p_AP_scores) / len(p_AP_scores)
e_ave_precision_over_queries = sum(e_AP_scores) / len(e_AP_scores)

print("The average precision across the queries for the production algorithm is: %s" % p_ave_precision_over_queries)
print("The average precision across the queries for the experimental algorithm is: %s" % e_ave_precision_over_queries)

The average precision across the queries for the production algorithm is: 0.5405066666666667
The average precision across the queries for the experimental algorithm is: 0.5529599999999996


### Multi-graded Evaluation Measure 1 - Discounted Cummulative Gain at rank k. 

In [47]:
# Gain for each relevance label
relevance_gain_dict = {'HR': 5, 'R': 1, 'N': 0}

# Function that calculates DCG@5
def DCG_Rank_K(ranked_list):
    gain = 0
    discounted_gain = 0
    for i, item in enumerate(ranked_list):
        rel = relevance_gain_dict[item]
        discounted_gain += (2**rel - 1) /  (math.log2(i + 1 + 1))
    return discounted_gain


p_DCG_scores = []
e_DCG_scores = []

# Calculate DCG@5 for each algorithm on all queries
for pairs in sample_pairs:
    p_DCG_scores.append(DCG_Rank_K(pairs[0]))
    e_DCG_scores.append(DCG_Rank_K(pairs[1]))

# Calculate the average of the average precisions across queries
p_ave_DCG_over_queries = sum(p_DCG_scores) / len(p_DCG_scores)
e_ave_DCG_over_queries = sum(e_DCG_scores) / len(e_DCG_scores)

print("The average DCG@5 across the queries for the production algorithm is: %s" % p_ave_DCG_over_queries)
print("The average DCG@5 across the queries for the experimental algorithm is: %s" % e_ave_DCG_over_queries)   

The average DCG@5 across the queries for the production algorithm is: 30.801079264723917
The average DCG@5 across the queries for the experimental algorithm is: 30.60084005457479


### Multi-graded Evaluation Measure 2 - Rank Biased Precision with persistence parameter $\theta = 0.8$ 

In [48]:
# Gain for each relevance label
relevance_gain_dict = {'HR': 5, 'R': 1, 'N': 0}

# Function that calculates RBP
def calc_RBP(ranked_list):
    theta = 0.8
    expected_utility = 0
    for i, item in enumerate(ranked_list):
        k_rel = relevance_gain_dict[item]
        expected_utility += k_rel * pow(theta, (i+1)*(1-theta))
    return expected_utility

p_RBP_scores = []
e_RBP_scores = []

# Calculate RBP for each algorithm on all queries
for pairs in sample_pairs:
    p_RBP_scores.append(calc_RBP(pairs[0]))
    e_RBP_scores.append(calc_RBP(pairs[1]))

# Calculate the average of the average precisions across queries
p_ave_RBP_over_queries = sum(p_RBP_scores) / len(p_RBP_scores)
e_ave_RBP_over_queries = sum(e_RBP_scores) / len(e_RBP_scores)

print("The average RBP across the queries for the production algorithm is: %s" %p_ave_RBP_over_queries)
print("The average RBP across the queries for the experimental algorithm is: %s" %e_ave_RBP_over_queries)

The average RBP across the queries for the production algorithm is: 8.600087841113798
The average RBP across the queries for the experimental algorithm is: 8.610925003329037


# Step 3: Calculate the $\Delta measure$ (5 poins)

In [49]:
# Function to compare two lists of scores
def compare_lists(list_a, list_b):
    b_better_pairs = []
    for i, item in enumerate(list_a):
        if item < list_b[i]:
            #b_better_pairs.append((list_b[i], item))
            b_better_pairs.append(i)
    return b_better_pairs


# Average Precision Difference Measure
E_better_AP = compare_lists(p_AP_scores, e_AP_scores)
E_better_AP_Pairs = list(map((lambda x: sample_pairs[x]), E_better_AP))
#print(E_better_AP_Pairs[0:5])

# DCG@5 Difference Measure
E_better_DCG = compare_lists(p_DCG_scores, e_DCG_scores)
E_better_DCG_Pairs = list(map((lambda x: sample_pairs[x]), E_better_DCG))
#print(E_better_DCG_Pairs[0:5])

# RBP Difference Measure
E_better_RBP = compare_lists(p_RBP_scores, e_RBP_scores)
E_better_RBP_Pairs = list(map((lambda x: sample_pairs[x]), E_better_RBP))
#print(E_better_RBP_Pairs[0:5])

# Step 4: Implement Interleaving (15 points)

### Balanced Interleaving

In [50]:
# Function to interleave two ranked lists
def interleave_Balanced(list_a, list_b, interleave_length=10):
    # print("List A:",list_a)
    # print("List B:",list_b)
    interleaved = []
    k_a = 1
    k_b = 1

    a_first = random.randint(0, 100) % 2
    # print("B first" if a_first == 0 else "A first")

    # while (k_a <= (len(list_a)) and k_b <= len(list_b)):
    while (len(interleaved) < (len(list_a) + len(list_b))):
        if (k_a < k_b) or ((k_a == k_b) and a_first == 1):
            # if (list_a[k_a-1] not in interleaved):
            interleaved.append(list_a[k_a - 1])
            k_a += 1
        else:
            # if (list_b[k_b-1] not in interleaved):
            interleaved.append(list_b[k_b - 1])
            k_b += 1

            # print(interleaved, "\n")
    return interleaved


# Interleave all sample pairs
team_draft_interleaved = []
for pairs in sample_pairs:
    team_draft_interleaved.append(interleave_Balanced(pairs[0], pairs[1]))

print("Rankings from P: %s and Rankings from E: %s" % (sample_pairs[0][0], sample_pairs[0][1]))
print("Interleaved with Balanced interleaving gives: %s " % (team_draft_interleaved[0]))

Rankings from P: ('N', 'R', 'HR', 'R', 'HR') and Rankings from E: ('R', 'R', 'N', 'HR', 'R')
Interleaved with Balanced interleaving gives: ['N', 'R', 'R', 'R', 'HR', 'N', 'R', 'HR', 'HR', 'R'] 


### Probabilistic Interleaving

In [51]:
class Interleave:

    def soft_max(self, ranked_list, gamma):
        selection_probabilities = []
        normalizer = 0
        for i in range(len(ranked_list)):
            non_normalized_prob = 1 / (i + 1)**gamma
            normalizer += non_normalized_prob
            selection_probabilities.append(non_normalized_prob)
        selection_probabilities = map(lambda x: x / normalizer, selection_probabilities)
        return list(selection_probabilities)

    def interleave_probabilistic(self, list_a, list_b, rank_length=10, gamma=3, clicks=False, click_prob=0):
        # Control final list length, if total results are shorter than rank_length
        if((len(list_a) + len(list_b)) < rank_length):
            rank_length = len(list_a) if len(list_a) <= len(list_b) else len(list_b)        
        
        # The interleaved list to be presented to the user
        interleaved = []
        
        # Lists of indices of each algorithms document in the interleaved list 
        a_indices = []
        b_indices = []
        
        # Click counts for each algorithm
        a_clicks = 0
        b_clicks = 0

        # Calculate probabilities for each document in each list
        prob_a = self.soft_max(list_a, gamma)
        prob_b = self.soft_max(list_b, gamma)

        soft_max_selections_a = list(np.random.choice(list_a, len(list_a), p=prob_a, replace=False))
        soft_max_selections_b = list(np.random.choice(list_b, len(list_b), p=prob_b, replace=False))

        for i in range(rank_length):
            if(len(soft_max_selections_a) > 0 and len(soft_max_selections_b) > 0):
                choice = np.random.ranf()
                if(choice <= 0.5):
                    interleaved.append(soft_max_selections_a.pop(0))
                    a_indices.append(i)
                else:
                    interleaved.append(soft_max_selections_b.pop(0))
                    b_indices.append(i)

            else:
                # Now it stops if when of the lists run out. (Question)
                break


        if(clicks):
            # Winner = 1 if B wins, else 0
            winner = 0
            for i, doc in enumerate(interleaved):
                click = random.uniform(0, 1)
                if(click <= click_prob):
                    if(i in a_indices):
                        a_clicks += 1
                    else:
                        b_clicks += 1
            # Only if b strictly beats a (Question)
            if(b_clicks > a_clicks): 
                winner = 1

        if(clicks):
            # If there is a click model, only output the winner (Question)
            return winner
        else:
            return interleaved

probabilistic_interleaver = Interleave()

# Interleave all sample pairs
probabilistic_interleaved = []
for pairs in sample_pairs:
    probabilistic_interleaved.append(probabilistic_interleaver.interleave_probabilistic(pairs[0], pairs[1]))

print("Rankings from P: %s and Rankings from E: %s" % (sample_pairs[0][0], sample_pairs[0][1])) 
print("Interleaved with Probabilistic interleaving gives: %s " % (probabilistic_interleaved[0]))

Rankings from P: ('N', 'R', 'HR', 'R', 'HR') and Rankings from E: ('R', 'R', 'N', 'HR', 'R')
Interleaved with Probabilistic interleaving gives: ['N', 'HR', 'R', 'R', 'R', 'N', 'R', 'R', 'HR'] 


# Step 5: Implement User Clicks Simulation (25 points)

### Random Click Model (RCM)

In [52]:
# All documents can be clicked with the same probability

# The probability of being clicked is (number of clicks)/(number of documents shown)

# This is what the model has to learn. 
headers = ['SessionID', 'TimePassed', 'TypeOfAction', 'QueryID', 'RegionID', 
'Rank1_URLID', 
'Rank2_URLID', 
'Rank3_URLID', 
'Rank4_URLID', 
'Rank5_URLID', 
'Rank6_URLID', 
'Rank7_URLID', 
'Rank8_URLID', 
'Rank9_URLID', 
'Rank10_URLID']

data = pd.read_csv('YandexRelPredChallenge.txt', sep='\t', names = headers)

class Random_Click_Model:
    
    def __init__(self):
        self.click_prob = 0

    def random_click_model_learn(self, query_dataframe):
        query_count = 0
        click_prob_per_query = []

        for index, row in query_dataframe.iterrows():
            # Include session check
            if(row['TypeOfAction'] == 'Q'):
                query_count += 1
                documents_shown = row[5:15].count()
                click_count = 0
                rows_below = 1
                if((index + rows_below) >= len(query_dataframe.index)):
                    click_prob_per_query.append(click_count / documents_shown)
                    break
                next_row_type = query_dataframe.iloc[index + rows_below]['TypeOfAction']
                if(next_row_type == 'C'):
                    while(next_row_type == 'C'):
                        click_count += 1
                        rows_below += 1
                        if((index + rows_below) >= len(query_dataframe.index)):
                            break
                        else:
                            next_row_type = query_dataframe.iloc[index + rows_below]['TypeOfAction']
                    click_prob_per_query.append(click_count / documents_shown)
                else:
                    click_prob_per_query.append(click_count / documents_shown)
        average_click_prob = sum(click_prob_per_query) / len(click_prob_per_query)
        self.click_prob = average_click_prob

random_click_model = Random_Click_Model()
random_click_model.random_click_model_learn(data)

print("The RCM trained on the Yandex Log file, gives a click probability p of %s." % random_click_model.click_prob)
  

The RCM trained on the Yandex Log file, gives a click probability p of 0.13445559411.


### Simplified Dynamic Bayesian Network model (SDBN)

# Step 6: Simulate Interleaving Experiment (10 points)

In [53]:
winners = []
for pairs in sample_pairs:
    winners.append(probabilistic_interleaver.interleave_probabilistic(pairs[0], 
                                                                      pairs[1], 
                                                                      clicks=True, 
                                                                      click_prob=random_click_model.click_prob))
    
print("The proportion of E wins over P across the 1000 samples is: %s" % (sum(winners)/len(winners)))

winners_AP_compare = []
for pairs in E_better_AP_Pairs:
    winners.append(probabilistic_interleaver.interleave_probabilistic(pairs[0], 
                                                                      pairs[1], 
                                                                      clicks=True, 
                                                                      click_prob=random_click_model.click_prob))
    
print("The proportion of E wins over P across the samples where E has a better AP is: %s" % (sum(winners)/len(winners)))

# Implement further

The proportion of E wins over P across the 1000 samples is: 0.274
The proportion of E wins over P across the samples where E has a better AP is: 0.28504359490274983


# Step 7: Results and Analyis (25 points)